In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point

In [2]:
CBG = gpd.read_file('../data/PA/Census_Block_Groups_2010.geojson')

In [3]:
CBG.columns


Index(['OBJECTID', 'STATEFP10', 'COUNTYFP10', 'TRACTCE10', 'BLKGRPCE10',
       'GEOID10', 'NAMELSAD10', 'MTFCC10', 'FUNCSTAT10', 'ALAND10', 'AWATER10',
       'INTPTLAT10', 'INTPTLON10', 'Shape__Area', 'Shape__Length', 'geometry'],
      dtype='object')

In [33]:
CBG.head()

,OBJECTID,STATEFP10,COUNTYFP10,TRACTCE10,BLKGRPCE10,GEOID10,NAMELSAD10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,Shape__Area,Shape__Length,geometry
0,1,42,101,010800,1,421010108001,Block Group 1,G5030,S,161887,0,+39.9687580,-075.1997251,1.742508e+06,8200.327170,"POLYGON ((-75.19851 39.96945, -75.19744 39.969..."
1,2,42,101,010800,2,421010108002,Block Group 2,G5030,S,103778,0,+39.9665475,-075.2004455,1.117026e+06,4364.980144,"POLYGON ((-75.19783 39.96571, -75.20006 39.965..."
2,3,42,101,010900,2,421010109002,Block Group 2,G5030,S,43724,0,+39.9642929,-075.1896435,4.706347e+05,3048.109084,"POLYGON ((-75.18766 39.96450, -75.18755 39.963..."
3,4,42,101,011000,2,421010110002,Block Group 2,G5030,S,108966,0,+39.9753585,-075.2113476,1.172871e+06,5169.004282,"POLYGON ((-75.20984 39.97351, -75.21221 39.973..."
4,5,42,101,011000,1,421010110001,Block Group 1,G5030,S,142244,0,+39.9724202,-075.2051689,1.531076e+06,10476.574129,"POLYGON ((-75.19855 39.97330, -75.19854 39.973..."


In [26]:
# 创建一个空字典来存储相邻区域信息
adjacent_regions = {}

# 遍历每个多边形
for index, row in CBG.iterrows():
    # 获取当前多边形的ID和几何图形
    polygon_id = row['OBJECTID']
    polygon = row['geometry']
    
    # 创建一个空列表来存储相邻区域的ID
    adjacent_ids = []
    
    # 遍历所有其他多边形
    for other_index, other_row in CBG.iterrows():
        # 排除与自身比较的情况
        if other_index != index:
            # 获取其他多边形的ID和几何图形
            other_polygon_id = other_row['OBJECTID']
            other_polygon = other_row['geometry']
            
            # 检查当前多边形是否与其他多边形相邻
            if polygon.touches(other_polygon) or polygon.intersects(other_polygon):
                adjacent_ids.append(other_polygon_id)
    
    # 将相邻区域的ID列表添加到字典中
    adjacent_regions[polygon_id] = adjacent_ids


In [27]:
# calculate the adjacency matrix
adjacency_matrix = np.zeros((len(CBG), len(CBG)))
# 遍历每个多边形及其相邻多边形列表
for polygon_id, adjacent_ids in adjacent_regions.items():
    # 获取当前多边形在邻接矩阵中的索引
    i = polygon_id - 1
    
    # 将相邻多边形在邻接矩阵中的对应位置设置为1
    for adjacent_id in adjacent_ids:
        j = adjacent_id - 1
        adjacency_matrix[i, j] = 1

# 打印邻接矩阵
print(adjacency_matrix)

[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 1. 0.]]


In [7]:
adjacency_matrix.shape

(1336, 1336)

In [3]:
from math import radians, sin, cos, sqrt, atan2

# DEFINE THE RADIUS OF THE EARTH IN KM
R = 6371.0

# Calculate the distance between two points
def dist(point1, point2):
    # Convert latitude and longitude to radians
    lat1_rad = float(point1['INTPTLAT10'])
    lon1_rad = float(point1['INTPTLON10'])
    lat2_rad = float(point2['INTPTLAT10'])
    lon2_rad = float(point2['INTPTLON10'])
    
    # Calculate the difference between the two points
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    
    # Use the Haversine formula to calculate the distance
    a = sin(dlat / 2) ** 2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    
    return distance

In [28]:
# calculate the distance matrix
distance_matrix = np.zeros((len(CBG), len(CBG)))
# go through adjacent regions
for i in range(len(CBG)):
    for j in range(len(CBG)):
        # only calculate the distance if the regions are adjacent
        if adjacency_matrix[i, j] == 1:
            distance_matrix[i, j] = dist(CBG.iloc[i], CBG.iloc[j])
# print the distance matrix
print(distance_matrix)

[[ 0.         14.38850595  0.         ...  0.          0.
   0.        ]
 [14.38850595  0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
  21.91867011]
 [ 0.          0.          0.         ...  0.         21.91867011
   0.        ]]


In [43]:
# show the max in matrix
def show_max_in_matrix(matrix):
    count = 0
    sum = 0
    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            if matrix[i][j] > 160:
                count += 1
            if matrix[i][j] > 0:
                sum += 1
    print(count, sum)
show_max_in_matrix(distance_matrix)

86 8390


In [11]:
# save the distance matrix
distance_matrix = pd.DataFrame(distance_matrix)
distance_matrix.to_csv("../data/PA/adjancency_matrix.csv")

In [5]:
bike_demand = pd.read_csv('../data/PA/share_bike_CBG/demand2019to2020CBG.csv', index_col=0)

In [6]:
bike_demand_region = bike_demand['demand_index'].unique()
len(bike_demand_region)

147

In [7]:
poi_embeddings = pd.read_csv('../data/PA/poi_embeddings_CBG.csv')

In [8]:
poi_embeddings_region = poi_embeddings['CBGID'].unique()
len(poi_embeddings_region)

986

In [9]:
# get the same element in poi_embeddings_region and bike_demand_region (they are 2 arrays)
Adjacent_ID = []
for val in bike_demand_region:
    if val in poi_embeddings_region:
        Adjacent_ID.append(val)
print(len(Adjacent_ID))

136


In [10]:
# save Adjacent_ID
filename = "../data/train/adjacent_id.txt"
with open(filename, "w") as file:
    for val in Adjacent_ID:
        file.write(str(val) + "\n")

In [11]:
filename = "../data/train/adjacent_id.txt"

with open(filename, "r") as file:
    adjacent_id = [int(line.strip()) for line in file]

In [12]:
Adjacency_Matrix = {}
for i in Adjacent_ID:
    for j in Adjacent_ID:
        key = (i, j)
        if i == j :
            Adjacency_Matrix[key] = 0
        else:
            dis = dist(CBG.iloc[i], CBG.iloc[j])
            if dis < 160:
                Adjacency_Matrix[key] = dist(CBG.iloc[i], CBG.iloc[j])
            else:
                Adjacency_Matrix[key] = 0

In [55]:
Adjacency_Matrix

{(10, 10): 0,
 (10, 53): 389.83190462349376,
 (10, 67): 299.44736634046757,
 (10, 122): 358.7085269878866,
 (10, 123): 398.83038973040254,
 (10, 124): 204.57154005160547,
 (10, 125): 215.78785219896815,
 (10, 126): 366.1580521293618,
 (10, 129): 318.1839303143677,
 (10, 130): 364.39168032130743,
 (10, 131): 281.4920913862582,
 (10, 132): 465.1544171494247,
 (10, 150): 250.4158816452341,
 (10, 152): 311.3093855971637,
 (10, 153): 280.71996871450784,
 (10, 154): 262.5647842487524,
 (10, 156): 248.82243747902913,
 (10, 157): 247.65739102158224,
 (10, 168): 240.68409062526507,
 (10, 197): 285.4876967322604,
 (10, 198): 263.73197491089905,
 (10, 199): 172.08082765095534,
 (10, 200): 300.3497147105745,
 (10, 201): 272.8108431717189,
 (10, 203): 304.7232723035129,
 (10, 206): 282.7680060005056,
 (10, 208): 431.0443834378896,
 (10, 210): 524.2704945076472,
 (10, 215): 394.0959880594069,
 (10, 216): 436.2322039292734,
 (10, 235): 360.2399986012155,
 (10, 236): 313.1551726403233,
 (10, 245): 226

In [13]:
Adjacency_matrix = np.zeros((len(Adjacent_ID),len(Adjacent_ID)))
for i in Adjacent_ID:
    for j in Adjacent_ID:
        i_index = Adjacent_ID.index(i)
        j_index = Adjacent_ID.index(j)
        if i == j :
            Adjacency_matrix[i_index,j_index] = 0
        else:
            dis = dist(CBG.iloc[i], CBG.iloc[j])
            if dis < 160:
                Adjacency_matrix[i_index,j_index] = dis
            else:
                Adjacency_matrix[i_index,j_index] = 0

In [62]:
# check the Adjacency matrix is fully connected
def is_graph_connected(adjacency_matrix, start_node):
    visited = set()
    dfs(adjacency_matrix, start_node, visited)
    return len(visited) == np.sqrt(len(adjacency_matrix))

def dfs(adjacency_matrix, node, visited):
    visited.add(node)
    for neighbor in adjacency_matrix:
        if neighbor[0] == node and neighbor[1] not in visited:
            dfs(adjacency_matrix, neighbor[1], visited)
        elif neighbor[1] == node and neighbor[0] not in visited:
            dfs(adjacency_matrix, neighbor[0], visited)

In [65]:
adjacency_matrix = [(i, j, weight) for (i, j), weight in Adjacency_Matrix.items()]
start_node = 10  # 选择一个起始节点
is_connected = is_graph_connected(adjacency_matrix, start_node)
print(is_connected)

True


In [ ]:
np.save('../data/train/adj_matrix.npy', Adjacency_matrix)

In [68]:
# save the adjacency matrix
import pickle
with open('../data/train/adj_matrix.pkl', 'wb') as f:
    pickle.dump(adjacency_matrix, f)

The Graph is fully connected: True